In [1]:
# http://www.swig.org/download.html
#  - this is code generator

# pip install pocketsphinx





In [11]:
import pyaudio
import wave
import sys

CHUNK = 1024

def play_stream(wave_stream):
    p = pyaudio.PyAudio()
    out_stream = p.open(format=p.get_format_from_width(wave_stream.getsampwidth()),
                channels=wave_stream.getnchannels(),
                rate=wave_stream.getframerate(),
                output=True)
    data = wave_stream.readframes(CHUNK)
    while data:
        out_stream.write(data)
        data = wave_stream.readframes(CHUNK)
    out_stream.stop_stream()
    out_stream.close()
    p.terminate()

def record_to_file(filename, seconds=5):
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 22050
    RECORD_SECONDS = seconds

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    print("Start...")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("...End")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [14]:
# yandex speech kit
api_key = "9d5ab675-adf6-48b6-a172-00a47530a614"
filename = "C:/dev/request.wav"
record_to_file(filename)
play_stream(wave.open(filename, 'rb'))

Start...
...End


In [34]:
import requests
import uuid
from bs4 import BeautifulSoup

UUID = str(uuid.uuid4()).replace("-", "")

def ask_yandex(wave_file, uid):
    # https://tech.yandex.ru/speechkit/cloud/doc/guide/concepts/asr-http-request-docpage/
    
    url = "https://asr.yandex.net/asr_xml?uuid={}&key={}&topic={}&lang={}&disableAntimat={}"
    url = url.format(uid, api_key, "queries", "ru-RU", "true")
    data = open(wave_file, "rb").read()
    headers = { 'Content-Type' : 'audio/x-wav', 'Content-Length' : str(len(data)) }
    resp = requests.post(url, data=data, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")    
    result = dict((var.string, float(var['confidence']))   
                  for var 
                  in dom.html.body.recognitionresults.findAll("variant"))
    
    return result
    
res = ask_yandex(filename, UUID)
print(res)

e024a623e7e4438aa412feddbb577041
<?xml version="1.0" encoding="utf-8"?><html><body><recognitionresults success="1">
<variant confidence="1">сколько сейчас времени</variant>
</recognitionresults>
</body></html>
{'сколько сейчас времени': 1.0}


In [37]:
query = "How much is the fish"

tts_url = "https://tts.voicetech.yandex.net/generate" + \
        "?key={}&text={}&format={}&quality={}&lang={}&speaker={}&speed=1.0&emotion={}"
tts_url = tts_url.format(api_key, query, "wav", "lo", "en-US", "jane", "evil")
resp = requests.get(tts_url)

In [47]:
with open("C:/dev/response.wav", "wb") as f:
    f.write(resp.content)
play_stream(wave.open("C:/dev/response.wav", 'rb'))